# EDA (Exploratory Data Analysis)
* Dataset from the [CanYouReadIt?](https://canyoureadit.com) project
* The survey was conducted starting from **/4/20/2021.** This is an ongoing survey, but the current data has been exported from the online database at **1/8/2022.**

In [1]:
try:
    import pandas as pd
    print(f"Pandas version: {pd.__version__} found!")
except ModuleNotFoundError:
    print("Install Pandas with: conda install pandas")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import datetime as dt
import matplotlib.font_manager
FONT_SIZE = 24
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['figure.figsize'] = (20,8)
plt.rcParams['grid.color'] = '#333333'
plt.rcParams['grid.linestyle'] = 'dotted'
plt.rcParams['grid.linewidth'] = 1
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = FONT_SIZE
plt.rcParams['legend.fontsize'] = FONT_SIZE
plt.rcParams['xtick.labelsize'] = FONT_SIZE
plt.rcParams['ytick.labelsize'] = FONT_SIZE
%config InlineBackend.figure_format ='retina'

Pandas version: 1.3.4 found!


In [2]:
from constants import (SAVEPATH, T)

In [5]:
df = pd.read_csv(SAVEPATH,
                 index_col=0,
                 sep=',',
                 parse_dates = ["create_dt"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34053 entries, 0 to 24598
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   image_id       34053 non-null  int64         
 1   create_dt      34053 non-null  datetime64[ns]
 2   session_id     34053 non-null  int64         
 3   mobile_user    34053 non-null  bool          
 4   mobile_image   34053 non-null  bool          
 5   response_yes   34053 non-null  bool          
 6   response_no    34053 non-null  bool          
 7   response_skip  34053 non-null  bool          
dtypes: bool(5), datetime64[ns](1), int64(2)
memory usage: 1.2 MB


In [6]:
df

,image_id,create_dt,session_id,mobile_user,mobile_image,response_yes,response_no,response_skip
0,6,2021-04-20 20:23:08,0,False,False,True,False,False
1,4,2021-04-21 11:33:17,1,False,False,True,False,False
2,9,2021-04-21 11:33:19,1,False,False,True,False,False
3,5,2021-04-21 12:42:57,1,False,False,True,False,False
4,11,2021-04-21 12:43:07,1,False,False,True,False,False
...,...,...,...,...,...,...,...,...
9456,384,2022-01-08 00:36:32,29643,True,True,False,False,True
9457,92,2022-01-08 00:42:51,29644,True,True,False,False,True
24596,452,2022-01-08 00:42:56,29645,False,False,False,False,True
24597,513,2022-01-08 00:52:31,29646,False,False,False,False,True
